In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
market_ticker = '^GSPC'
st = 'f'

stime='2018-01-01'
ftime=None

In [5]:
def get_data_from_ticker(tick, stime, ftime, interval='1d'):
    ticker = yf.Ticker(tick)
    # df = ticker.history(start='2021-01-01', end='2022-06-30')
    # df = ticker.history(start='2021-01-01', end='2022-07-26')
    # df = ticker.history(start='2020-01-01', end='2021-01-01')
    # df = ticker.history(start='2022-05-28')
    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = pd.DataFrame(get_data_from_ticker(market_ticker, stime, ftime))
# sf = pd.DataFrame(get_data_from_ticker(st, stime, ftime))
# gf[st] = sf[st]/


df.head(3)

,^GSPC
Date,
2018-01-02,2695.810059
2018-01-03,2713.060059
2018-01-04,2723.989990


In [6]:
def add_percent_update(tick):
    """ date stock's or market's update"""
    gf_copy = df.copy(deep=True)
    gf_copy = pd.DataFrame(gf_copy.drop(df.index[0]))
    gf_copy_v = gf_copy[tick].values
    gf_v = df.copy(deep=True)
    gf_vv = gf_v[tick].values

    for i in range(len(gf_copy)):
        gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

    gf_vv = np.insert(gf_vv, 0, 0)
    gf_vv = np.delete(gf_vv, -1)
    return gf_vv

df[f"% update {market_ticker}"] = add_percent_update(market_ticker)
# gf[f"% update {st}"] = add_percent_update(st)

df.head(3)

,^GSPC,% update ^GSPC
Date,,
2018-01-02,2695.810059,0.000000
2018-01-03,2713.060059,0.639882
2018-01-04,2723.989990,0.402864


In [7]:
r = list(df['% update ^GSPC'].values)

for i in range(len(r)):
    if df.iloc[i, 1] > 0:
        r[i] = 'Plus'
    else:
        r[i] = 'Minus'
        
df['Side'] = r        

Signal module

In [8]:
''' easy signal way'''

# r = list(df['% update ^GSPC'].values)

# for i in range(len(r)):
#     try:
#         if df.iloc[i, 1] > 0:
#             r[i+1] = 'Long'
#         else:
#             r[i+1] = 'Short'
#     except:
#         pass
        
# df['Signal'] = r



' easy signal way'

In [9]:
''' more or less '''


# ''' получаем точки разворота '''
# r = list(df['% update ^GSPC'].values)

# for i in range(len(r)):
#     try:
#         if df.iloc[i, 1] < 0 and df.iloc[i-1, 1] < 1:
#             r[i+1] = 'Long'
#         elif df.iloc[i, 1] > 0 and df.iloc[i-1, 1] > -1:
#             r[i+1] = 'Short'
#         else:
#             pass
#     except:
#         pass
    
# df['Signal'] = r

' more or less '

MA tests

In [10]:
# ''' MA tests 
# if Fast more or less than slow
# True     656
# False    599
# '''


# r = list(df['% update ^GSPC'].values)
# df['Signal'] = r

# df['ma_fast'] = df['^GSPC'].ewm(span=12, adjust=False).mean()
# df['ma_slow'] = df['^GSPC'].ewm(span=26, adjust=False).mean()



# for i in range(len(r)):
#     try:
#         if df.iloc[i, 5] < df.iloc[i, 4]:
#             r[i+1] = 'Long'
#         elif df.iloc[i, 5] > df.iloc[i, 4]:
#             r[i+1] = 'Short'
#         else:
#             pass
#     except:
#         pass

# df['Signal'] = r

In [11]:
''' MA tests 
if slow or fast up or down
SLOW (26)
    True     657
    False    598
    
FAST (12)
    True     644
    False    611

if FAST and FAST-1 more or less then price and price-1
MA 50
True     630
False    574

MA 150
True     594
False    510

MA 35
True     630
False    574
13
    
if price more or less MA
MA 50    
True     629
False    575

MA 40    
True     637
False    577

MA 35      !!!
True     641
False    578
19


MA 30
True     642
False    582

F35 S100
True     618
False    536

F35 S80
True     617
False    557
-19

F35 S130 more/less price  !!!
True     607
False    517
25

F35 S150 more/less price  !!!
True     607
False    517
20

F35 S130 more/less price + fast with price and price-1!!!
True     607
False    517
31

F35 S130 more/less price + fast and slow with price and price-1!!!
True     607
False    517
34.6

F35 S130 more/less price + fast with price 0, -1, -2 and slow with price 0, -1!!!
True     607
False    517
35.9

F20 S130 more/less price + fast with price 0, -1, -2 and slow with price 0, -1!!!
True     607
False    517
35.9
36.7
'''
# fma = 20
# mma = 50
# sma = 130


# r = list(df['% update ^GSPC'].values)
# df['Signal'] = r

# df['ma_fast'] = df['^GSPC'].ewm(span=fma, adjust=False).mean()
# df['ma_slow'] = df['^GSPC'].ewm(span=sma, adjust=False).mean()
# # df['ma_middle'] = df['^GSPC'].ewm(span=sma, adjust=False).mean()


# for i in range(len(r)):
#     try:
   
    #     if i < sma:
    #         r[i] = None
    #     elif df.iloc[i, 0] > df.iloc[i, 4] \
    #         and df.iloc[i-1, 0] > df.iloc[i-1, 4] \
    #         and df.iloc[i-2, 0] > df.iloc[i-2, 4]\
    #         and df.iloc[i, 0] > df.iloc[i, 5] \
    #         and df.iloc[i-1, 0] > df.iloc[i-1, 5]:
    #         r[i+1] = 'Long'
    #     elif df.iloc[i, 0] < df.iloc[i, 4] \
    #         and df.iloc[i-1, 0] < df.iloc[i-1, 4] \
    #         and df.iloc[i-2, 0] < df.iloc[i-2, 4] \
    #         and df.iloc[i, 0] < df.iloc[i, 5] \
    #         and df.iloc[i-1, 0] < df.iloc[i-1, 5]:
    #         r[i+1] = 'Short'
    #     else:
    #         pass
    # except:
    #     pass

# df['Signal'] = r
# df

' MA tests \nif slow or fast up or down\nSLOW (26)\n    True     657\n    False    598\n    \nFAST (12)\n    True     644\n    False    611\n\nif FAST and FAST-1 more or less then price and price-1\nMA 50\nTrue     630\nFalse    574\n\nMA 150\nTrue     594\nFalse    510\n\nMA 35\nTrue     630\nFalse    574\n13\n    \nif price more or less MA\nMA 50    \nTrue     629\nFalse    575\n\nMA 40    \nTrue     637\nFalse    577\n\nMA 35      !!!\nTrue     641\nFalse    578\n19\n\n\nMA 30\nTrue     642\nFalse    582\n\nF35 S100\nTrue     618\nFalse    536\n\nF35 S80\nTrue     617\nFalse    557\n-19\n\nF35 S130 more/less price  !!!\nTrue     607\nFalse    517\n25\n\nF35 S150 more/less price  !!!\nTrue     607\nFalse    517\n20\n\nF35 S130 more/less price + fast with price and price-1!!!\nTrue     607\nFalse    517\n31\n\nF35 S130 more/less price + fast and slow with price and price-1!!!\nTrue     607\nFalse    517\n34.6\n\nF35 S130 more/less price + fast with price 0, -1, -2 and slow with price 

In [12]:
''' MACD tests 


if macd_signal more or less 0 
True     657
False    598

if macd more or less 0 
True     656
False    599
12

if macd_signal more or less then previous macd
False    636
True     619

if macd more or less then previous macd
True     634
False    621

if macd and macd_signal more or less then previous 
False    636
True     619

if macd more or less macd_signal
True     637
False    618

if macd more or less 0, and macd_signal conversly
True   638
False  617


if macd more or less -1,-2
False    621
True     607
30.5

if macd_signal more or less -1,-2
False    625
True     603
19.9

if macd more or less 0 and macd_signal-1 conversly + macd_signal conversly macd
True     602
False    523
43.57497817317594
'''


fma = 12
sma = 26


r = list(df['% update ^GSPC'].values)
df['Signal'] = r

df['ma_fast'] = df['^GSPC'].ewm(span=fma, adjust=False).mean()
df['ma_slow'] = df['^GSPC'].ewm(span=sma, adjust=False).mean()
df['macd'] = df['ma_fast'] - df['ma_slow']
df['signal_macd'] = df['macd'].ewm(span=9, adjust=False).mean()



for i in range(len(r)):
    try:
        ''' macd '''
        if i < sma:
            r[i] = None
        elif df.iloc[i, 6] > 0 and df.iloc[i-1, 6] < 0 and df.iloc[i, 7] > 0:
            r[i+1] = 'Long'
        elif df.iloc[i, 6] < 0 and df.iloc[i-1, 6] > 0 and df.iloc[i, 7] < 0:
            r[i+1] = 'Short'
        else:
            pass
        
    except:
        pass

df['Signal'] = r
df.tail(5)

,^GSPC,% update ^GSPC,Side,Signal,ma_fast,ma_slow,macd,signal_macd
Date,,,,,,,,
2022-12-19,3817.659912,-0.900752,Minus,-0.900752,3932.249321,3934.938535,-2.689214,22.659746
2022-12-20,3821.620117,0.103734,Plus,0.103734,3915.229443,3926.544579,-11.315135,15.864769
2022-12-21,3878.439941,1.486799,Plus,1.486799,3909.569520,3922.981272,-13.411752,10.009465
2022-12-22,3822.389893,-1.445170,Minus,-1.44517,3896.157270,3915.530059,-19.372789,4.133014
2022-12-23,3844.820068,0.586810,Plus,0.58681,3888.259239,3910.292282,-22.033043,-1.100197


Закрытие пробелов 

In [13]:
''' заполняем пробелы '''
r = list(df['Signal'].values)


for i in range(len(r)):
    if df.iloc[i, 3] == None:
        pass
    elif type(df.iloc[i, 3]) == str:
        if df.iloc[i, 3] == 'Long' or df.iloc[i, 3] == 'Short':
            pass 
        else:
            df.iloc[i, 3] = df.iloc[i-1, 3]
    else:
        df.iloc[i, 3] = df.iloc[i-1, 3]
        

df.tail(10)

,^GSPC,% update ^GSPC,Side,Signal,ma_fast,ma_slow,macd,signal_macd
Date,,,,,,,,
2022-12-12,3990.560059,1.427930,Plus,Long,3976.693631,3947.687927,29.005704,42.178492
2022-12-13,4019.649902,0.728966,Plus,Long,3983.302288,3953.018444,30.283845,39.799563
2022-12-14,3995.320068,-0.605272,Minus,Long,3985.151177,3956.151897,28.999280,37.639506
2022-12-15,3895.750000,-2.492168,Minus,Long,3971.397150,3951.677683,19.719467,34.055498
2022-12-16,3852.360107,-1.113775,Minus,Long,3953.083759,3944.320825,8.762934,28.996986
2022-12-19,3817.659912,-0.900752,Minus,Long,3932.249321,3934.938535,-2.689214,22.659746
2022-12-20,3821.620117,0.103734,Plus,Long,3915.229443,3926.544579,-11.315135,15.864769
2022-12-21,3878.439941,1.486799,Plus,Long,3909.569520,3922.981272,-13.411752,10.009465
2022-12-22,3822.389893,-1.445170,Minus,Long,3896.157270,3915.530059,-19.372789,4.133014


Result module

In [14]:
r = list(df['% update ^GSPC'].values)

for i in range(len(r)):
    try:
        if df.iloc[i, 2] == 'Minus' and df.iloc[i, 3] == 'Short':
            r[i] = True
        elif df.iloc[i, 2] == 'Plus' and df.iloc[i, 3] == 'Long':    
            r[i] = True
        elif df.iloc[i, 2] == 'Minus' and df.iloc[i, 3] == 'Long':    
            r[i] = False  
        elif df.iloc[i, 2] == 'Plus' and df.iloc[i, 3] == 'Short':    
            r[i] = False 
        else:
            r[i] = None
    except:
        pass
    
df['Result'] = r
df['Result'].value_counts()
df.tail(10)

,^GSPC,% update ^GSPC,Side,Signal,ma_fast,ma_slow,macd,signal_macd,Result
Date,,,,,,,,,
2022-12-12,3990.560059,1.427930,Plus,Long,3976.693631,3947.687927,29.005704,42.178492,True
2022-12-13,4019.649902,0.728966,Plus,Long,3983.302288,3953.018444,30.283845,39.799563,True
2022-12-14,3995.320068,-0.605272,Minus,Long,3985.151177,3956.151897,28.999280,37.639506,False
2022-12-15,3895.750000,-2.492168,Minus,Long,3971.397150,3951.677683,19.719467,34.055498,False
2022-12-16,3852.360107,-1.113775,Minus,Long,3953.083759,3944.320825,8.762934,28.996986,False
2022-12-19,3817.659912,-0.900752,Minus,Long,3932.249321,3934.938535,-2.689214,22.659746,False
2022-12-20,3821.620117,0.103734,Plus,Long,3915.229443,3926.544579,-11.315135,15.864769,True
2022-12-21,3878.439941,1.486799,Plus,Long,3909.569520,3922.981272,-13.411752,10.009465,True
2022-12-22,3822.389893,-1.445170,Minus,Long,3896.157270,3915.530059,-19.372789,4.133014,False


In [15]:
r = list(df['% update ^GSPC'].values)

for i in range(len(r)):
    try:
        if df.iloc[i, 3] == 'Short':
            r[i] = (df.iloc[i, 1] * -1)
        elif df.iloc[i, 3] == 'Long':
            r[i] = df.iloc[i, 1]
        else:
            r[i] = 0
    except:
        pass
    
df['Sum'] = r
df['Sum'].sum()

43.57497817317594